In [7]:
import requests
from bs4 import BeautifulSoup
import os


page_urls = [
    "https://www.gutenberg.org/ebooks/bookshelf/20", 
    "https://www.gutenberg.org/ebooks/bookshelf/20?start_index=26", 
    "https://www.gutenberg.org/ebooks/bookshelf/20?start_index=51", 
    "https://www.gutenberg.org/ebooks/bookshelf/20?start_index=76", 
]

for url in page_urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    book_links = soup.select('li.booklink a')
    os.makedirs("books", exist_ok=True)
    for link in book_links:
        title = link.text.strip().replace('/', '-').replace('\n', ' ').replace('\r', '')
        book_id = link['href'].split("/")[-1]
        book_url = f"https://www.gutenberg.org/files/{book_id}/{book_id}-h/{book_id}-h.htm"
        book_response = requests.get(book_url)
        book_content = book_response.content
        book_soup = BeautifulSoup(book_content, 'html.parser')
        book_text = book_soup.find('body').get_text()
        filename = f"books/{title}.txt"
        with open(filename, "w", encoding='utf-8') as file:
            file.write(book_text)

        print(f"{title} to {filename}")

Alice's Adventures in Wonderland Lewis Carroll 30609 downloads to books/Alice's Adventures in Wonderland Lewis Carroll 30609 downloads.txt
Anne of Green Gables L. M. Montgomery 9175 downloads to books/Anne of Green Gables L. M. Montgomery 9175 downloads.txt
Treasure Island Robert Louis Stevenson 9038 downloads to books/Treasure Island Robert Louis Stevenson 9038 downloads.txt
A Christmas Carol in Prose; Being a Ghost Story of Christmas Charles Dickens 8467 downloads to books/A Christmas Carol in Prose; Being a Ghost Story of Christmas Charles Dickens 8467 downloads.txt
The Wonderful Wizard of Oz L. Frank Baum 8360 downloads to books/The Wonderful Wizard of Oz L. Frank Baum 8360 downloads.txt
Peter Pan J. M. Barrie 7701 downloads to books/Peter Pan J. M. Barrie 7701 downloads.txt
Little Women Louisa May Alcott 7685 downloads to books/Little Women Louisa May Alcott 7685 downloads.txt
The Jungle Book Rudyard Kipling 5394 downloads to books/The Jungle Book Rudyard Kipling 5394 downloads.tx

Rose in Bloom Louisa May Alcott 347 downloads to books/Rose in Bloom Louisa May Alcott 347 downloads.txt
The Pink Fairy Book Andrew Lang 343 downloads to books/The Pink Fairy Book Andrew Lang 343 downloads.txt
The Light Princess George MacDonald 341 downloads to books/The Light Princess George MacDonald 341 downloads.txt
The Phoenix and the Carpet E. Nesbit 319 downloads to books/The Phoenix and the Carpet E. Nesbit 319 downloads.txt
Flower Fables Louisa May Alcott 315 downloads to books/Flower Fables Louisa May Alcott 315 downloads.txt
The Adventures of Reddy Fox Thornton W. Burgess 302 downloads to books/The Adventures of Reddy Fox Thornton W. Burgess 302 downloads.txt
Rebecca of Sunnybrook Farm Kate Douglas Smith Wiggin 297 downloads to books/Rebecca of Sunnybrook Farm Kate Douglas Smith Wiggin 297 downloads.txt
Tanglewood Tales Nathaniel Hawthorne 290 downloads to books/Tanglewood Tales Nathaniel Hawthorne 290 downloads.txt
Kilmeny of the Orchard L. M. Montgomery 288 downloads to b

In [9]:
import re
import os
import chardet

# spécifier le chemin du dossier contenant les fichiers
folder_path = 'books'

# boucler à travers tous les fichiers dans le dossier
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'rb') as f:
            # détecter l'encodage de caractères du fichier
            result = chardet.detect(f.read())
            encoding = result['encoding']
            
        with open(file_path, 'r', encoding=encoding) as f:

            # lire le contenu du fichier
            text = f.read()
            
            # supprimer les URLs
            text = re.sub(r'http\S+', '', text)
            
            # supprimer la ponctuation
            text = re.sub(r'[^\w\s]', '', text)
            
            # supprimer les nombres
            text = re.sub(r'\d+', '', text)
            
            # remplacer les espaces non entre les mots par une nouvelle ligne
            text = re.sub(r'(?<!\w)\s+|\s+(?!\w)', '\n', text)
            
            # supprimer les lignes vides ou qui ne contiennent pas de texte
            text = '\n'.join(line.strip() for line in text.split('\n') if line.strip())
            
            # supprimer les mots "pg"
            text = re.sub(r'\b pg\b', '', text)
            
            # supprimer les caractères spéciaux
            text = re.sub(r'[^\w\s]', '', text)
            
            # supprimer les lignes contenant "CONTENTS" ou "contents" ou "chapter" + chiffre romain ou "CHAPTER" + chiffre romain
            text = re.sub(r'^(CONTENTS|Contents|contents|chapter\s+[IVXLCM]+|CHAPTER\s+[IVXLCM]+Chapter\s+[IVXLCM]+).*$\n?', '', text, flags=re.MULTILINE)
     
        # écrire le texte traité dans le même fichier
        with open(file_path, 'w', encoding=encoding) as f:
            f.write(text)


In [10]:
import re
import os
import chardet

# spécifier le chemin du dossier contenant les fichiers
folder_path = 'books'

# boucler à travers tous les fichiers dans le dossier
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'rb') as f:
            # détecter l'encodage de caractères du fichier
            result = chardet.detect(f.read())
            encoding = result['encoding']

        # initialiser le compteur de lignes à 0
        line_count = 0

        with open(file_path, 'r', encoding=encoding) as f:
            # initialiser la variable pour stocker les lignes du fichier
            lines = []
            
            for line in f:
                
                if line:
                    line_count += 1
                    
                    if line_count > 100:
                        # ajouter la ligne à la liste des lignes
                        lines.append(line)
                
                    if line_count >= 200:
                        # si on a atteint le nombre maximum de lignes, sortir de la boucle
                        break

            # concaténer les lignes en un seul texte
            text = ''.join(lines)

        # écrire le texte traité dans le même fichier
        with open(file_path, 'w', encoding=encoding) as f:
            f.write(text)


In [6]:
'''import csv
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
import os

# Define the path to the directory where the books are stored
books_dir = "books"

# Define the path to the directory where the summaries will be stored
summaries_dir = "summaries"

# Create the directories if they don't exist
if not os.path.exists(books_dir):
    os.makedirs(books_dir)

if not os.path.exists(summaries_dir):
    os.makedirs(summaries_dir)

# Create the CSV file and write the header
with open("book_summaries.csv", "w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["text", "summary"])

    # Process each book in the directory
    for filename in os.listdir(books_dir):
        book_filepath = os.path.join(books_dir, filename)

        # Read the book text from the file
        with open(book_filepath, "r", encoding="ISO-8859-1") as f:
            text = f.read()

        # Summarize the text using the TextRank algorithm
        summarizer = TextRankSummarizer()
        tokenizer = Tokenizer("english")
        parser = PlaintextParser(text, tokenizer)
        summary = summarizer(parser.document, sentences_count=3)

        # Write the summary to a file with the same name as the original file
        summary_filename = os.path.splitext(filename)[0] + "_summary.txt"
        summary_filepath = os.path.join(summaries_dir, summary_filename)
        with open(summary_filepath, "w", encoding="utf-8") as summary_file:
            summary_file.write(str(summary))

        # Write the book text and summary text to the CSV file
        writer.writerow([text, str(summary)])

print("Done.")
'''

Done.


In [11]:
import csv
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
import os

# Define the path to the directory where the books are stored
books_dir = "books"

# Define the path to the directory where the summaries will be stored
summaries_dir = "summaries"

# Create the directories if they don't exist
if not os.path.exists(books_dir):
    os.makedirs(books_dir)

if not os.path.exists(summaries_dir):
    os.makedirs(summaries_dir)

# Create the CSV file and write the header
with open("book_summaries.csv", "w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["text", "summary"])

    # Process each book in the directory
    for filename in os.listdir(books_dir):
        book_filepath = os.path.join(books_dir, filename)

        # Read the book text from the file
        with open(book_filepath, "r", encoding="ISO-8859-1") as f:
            text = f.read()

        # Summarize the text using the TextRank algorithm
        summarizer = TextRankSummarizer()
        tokenizer = Tokenizer("english")
        parser = PlaintextParser(text, tokenizer)
        summary = summarizer(parser.document, sentences_count=3)

        # Write the summary to a file with the same name as the original file
        summary_filename = os.path.splitext(filename)[0] + "_summary.txt"
        summary_filepath = os.path.join(summaries_dir, summary_filename)
        with open(summary_filepath, "w", encoding="utf-8") as summary_file:
            summary_file.write(str(summary))

        # Write the book text and summary text to the CSV file
        writer.writerow([text.replace('\n', ' '), str(summary).replace('\n', ' ')])

print("Done.")


Done.


In [5]:
import os
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# chemin du dossier contenant les fichiers texte
directory = "summaries/"

# chemin du dossier où les fichiers résumés seront stockés
summary_directory = "summaries/summaries/"

# création du répertoire de stockage s'il n'existe pas
if not os.path.exists(summary_directory):
    os.makedirs(summary_directory)

# initialisation de la liste des stopwords
stop_words = set(stopwords.words('english'))

# boucle sur les fichiers dans le dossier "summaries"
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        # chemin complet du fichier texte
        filepath = os.path.join(directory, filename)
        
        # lecture du contenu du fichier
        with open(filepath, "r") as f:
            text = f.read()
        
        # tokenization des phrases et des mots
        sentences = sent_tokenize(text)
        words = word_tokenize(text)
        
        # suppression de la ponctuation et des stopwords
        words = [word.lower() for word in words if word not in string.punctuation and word.lower() not in stop_words]
        
        # création de la matrice TF-IDF
        vectorizer = TfidfVectorizer(norm=None)
        tf_idf_matrix = vectorizer.fit_transform(sentences)
        
        # calcul du score de pertinence des phrases
        sentence_scores = {}
        for i, sentence in enumerate(sentences):
            feature_scores = tf_idf_matrix.getcol(i).toarray()
            sentence_scores[i] = sum(feature_scores[j] for j in range(len(feature_scores)))
        
        # sélection des phrases les plus pertinentes pour le résumé
        summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:3]
        summary_sentences = sorted(summary_sentences)
        summary = ' '.join([sentences[i] for i in summary_sentences])
        
        # écriture du résumé dans un nouveau fichier
        summary_filename = os.path.splitext(filename)[0] + "_summary.txt"
        summary_filepath = os.path.join(summary_directory, summary_filename)
        with open(summary_filepath, "w") as f:
            f.write(summary)


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [6]:
import csv
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
import os

# Define the path to the directory where the books are stored
books_dir = "summaries"

# Define the path to the directory where the summaries will be stored
summaries_dir = "summariess"

# Create the directories if they don't exist
if not os.path.exists(books_dir):
    os.makedirs(books_dir)

if not os.path.exists(summaries_dir):
    os.makedirs(summaries_dir)

# Create the CSV file and write the header
with open("book_summaries.csv", "w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["text", "summary"])

    # Process each book in the directory
    for filename in os.listdir(books_dir):
        book_filepath = os.path.join(books_dir, filename)

        # Read the book text from the file
        with open(book_filepath, "r", encoding="ISO-8859-1") as f:
            text = f.read()

        # Summarize the text using the TextRank algorithm
        summarizer = TextRankSummarizer()
        tokenizer = Tokenizer("english")
        parser = PlaintextParser(text, tokenizer)
        summary = summarizer(parser.document, sentences_count=3)

        # Write the summary to a file with the same name as the original file
        summary_filename = os.path.splitext(filename)[0] + "_summary.txt"
        summary_filepath = os.path.join(summaries_dir, summary_filename)
        with open(summary_filepath, "w", encoding="utf-8") as summary_file:
            summary_file.write(str(summary))

        # Write the book text and summary text to the CSV file
        writer.writerow([text, str(summary)])

print("Done.")


PermissionError: [Errno 13] Permission denied: 'summaries\\summaries'

In [ ]:
 from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
import os

# Define the path to the directory where the books are stored
books_dir = "summaries"

# Define the path to the directory where the summaries will be stored
summaries_dir = "summariess"
    # Process each book in the directory
    for filename in os.listdir(books_dir):
        book_filepath = os.path.join(books_dir, filename)

        # Read the book text from the file
        with open(book_filepath, "r", encoding="ISO-8859-1") as f:
            text = f.read()

        # Summarize the text using the TextRank algorithm
        summarizer = TextRankSummarizer()
        tokenizer = Tokenizer("english")
        parser = PlaintextParser(text, tokenizer)
        summary = summarizer(parser.document, sentences_count=3)

        # Write the summary to a file with the same name as the original file
        summary_filename = os.path.splitext(filename)[0] + "_summary.txt"
        summary_filepath = os.path.join(summaries_dir, summary_filename)
        with open(summary_filepath, "w", encoding="utf-8") as summary_file:
            summary_file.write(str(summary))
